k-Folds Cross Validation helps to detect which models is the best applicable according to the mean accuracy score. fter applying the cross validation, the model with the highest mean accuracy is choosen to train the data by fitting it and look at the feature importance.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("white")

pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

In [2]:
med = pd.read_pickle("med_k_fold.pkl")

In [3]:
med.head()

,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,No_Show,Age_Normalized,Waiting_Days_Normalized,Handicap_1,Handicap_2,Handicap_3,Handicap_4,SMS_Received_1,Day_Of_Week_Monday,Day_Of_Week_Saturday,Day_Of_Week_Thursday,Day_Of_Week_Tuesday,Day_Of_Week_Wednesday
0,29872499824296,5642903,1,2016-04-29,2016-04-29,Jardim Da Penha,0,1,0,0,0,0.539130,0.0,0,0,0,0,0,0,0,0,0,0
1,558997776694438,5642503,0,2016-04-29,2016-04-29,Jardim Da Penha,0,0,0,0,0,0.486957,0.0,0,0,0,0,0,0,0,0,0,0
2,4262962299951,5642549,1,2016-04-29,2016-04-29,Mata Da Praia,0,0,0,0,0,0.539130,0.0,0,0,0,0,0,0,0,0,0,0
3,867951213174,5642828,1,2016-04-29,2016-04-29,Pontal De Camburi,0,0,0,0,0,0.069565,0.0,0,0,0,0,0,0,0,0,0,0
4,8841186448183,5642494,1,2016-04-29,2016-04-29,Jardim Da Penha,0,1,1,0,0,0.486957,0.0,0,0,0,0,0,0,0,0,0,0


### next steps:

    (Test set as new df = 20.000 observations, target split equally (manually)
    Rest = Train set as new df)
    
 
    1. med: make index to column using .reset_index = med_index
    2. create two subsets for 50% 50% split '0' and '1' -> med_index0, med_index1
        2.a make med_index0 and med_index1 to list of dicts, using df.to_dict(orient='records') =
        med_index_records
        2.b create test set -> using random.sample(med_index_records, size=20000/2)
        -> test_set0 and test_set1
        2.c concat both test sets
        2.d convert test set to df = med_test
        2.e shuffle med_test
    3. med.drop(med_test["index"].to_list())
    4. rename med to train
        4.a drop index column of train
    5. apply SMOTE on train
    6. 5 fold cross validation

In [4]:
med.No_Show.value_counts()

0    88207
1    22314
Name: No_Show, dtype: int64

In [5]:
med_index=med.reset_index()

In [6]:
med_index.head()

,index,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,No_Show,Age_Normalized,Waiting_Days_Normalized,Handicap_1,Handicap_2,Handicap_3,Handicap_4,SMS_Received_1,Day_Of_Week_Monday,Day_Of_Week_Saturday,Day_Of_Week_Thursday,Day_Of_Week_Tuesday,Day_Of_Week_Wednesday
0,0,29872499824296,5642903,1,2016-04-29,2016-04-29,Jardim Da Penha,0,1,0,0,0,0.539130,0.0,0,0,0,0,0,0,0,0,0,0
1,1,558997776694438,5642503,0,2016-04-29,2016-04-29,Jardim Da Penha,0,0,0,0,0,0.486957,0.0,0,0,0,0,0,0,0,0,0,0
2,2,4262962299951,5642549,1,2016-04-29,2016-04-29,Mata Da Praia,0,0,0,0,0,0.539130,0.0,0,0,0,0,0,0,0,0,0,0
3,3,867951213174,5642828,1,2016-04-29,2016-04-29,Pontal De Camburi,0,0,0,0,0,0.069565,0.0,0,0,0,0,0,0,0,0,0,0
4,4,8841186448183,5642494,1,2016-04-29,2016-04-29,Jardim Da Penha,0,1,1,0,0,0.486957,0.0,0,0,0,0,0,0,0,0,0,0


In [7]:
med_index0 = med_index[med_index['No_Show'] == 0]

med_index1 = med_index[med_index['No_Show'] == 1]

In [8]:
med_index0_records=med_index0.to_dict(orient='records')

med_index1_records=med_index1.to_dict(orient='records')

In [9]:
import random

test_set0=random.sample(med_index0_records, 10000)

test_set1=random.sample(med_index1_records, 10000)

In [10]:
test0=pd.DataFrame.from_dict(test_set0)

In [11]:
test1=pd.DataFrame.from_dict(test_set1)

In [12]:
med_test = pd.concat([test0, test1])

In [13]:
test = med_test.sample(frac=1)

In [14]:
test

,index,Patient_ID,Appointment_ID,Gender,Scheduled_Day,Appointment_Day,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,No_Show,Age_Normalized,Waiting_Days_Normalized,Handicap_1,Handicap_2,Handicap_3,Handicap_4,SMS_Received_1,Day_Of_Week_Monday,Day_Of_Week_Saturday,Day_Of_Week_Thursday,Day_Of_Week_Tuesday,Day_Of_Week_Wednesday
9777,35239,627449443436378,5734376,1,2016-05-24,2016-05-30,Tabuazeiro,0,0,0,0,1,0.139130,6.0,0,0,0,0,1,1,0,0,0,0
8906,109460,6325517594936,5748872,1,2016-05-31,2016-06-02,Inhanguetá,0,0,0,0,0,0.095652,2.0,0,0,0,0,0,0,0,1,0,0
342,28304,629253219269171,5552385,1,2016-04-06,2016-05-06,Mata Da Praia,0,0,0,0,1,0.608696,30.0,0,0,0,0,0,0,0,0,0,0
2326,97395,478574723942336,5759775,0,2016-06-01,2016-06-01,Do Moscoso,0,0,0,0,0,0.034783,0.0,0,0,0,0,0,0,0,0,0,1
8012,91745,35871196186,5765082,1,2016-06-02,2016-06-07,Nova Palestina,0,0,0,0,0,0.026087,5.0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,90275,2745156214988,5760690,1,2016-06-01,2016-06-08,República,0,0,0,0,0,0.130435,7.0,0,0,0,0,1,0,0,0,0,1
3742,61668,92388644259316,5679553,1,2016-05-10,2016-05-11,Cruzamento,1,1,0,0,1,0.513043,1.0,0,0,0,0,0,0,0,0,0,1
58,29365,548128696197394,5717243,1,2016-05-19,2016-05-31,Cruzamento,0,0,0,0,1,0.269565,12.0,0,0,0,0,1,0,0,0,1,0
6043,21940,589893676145,5682723,1,2016-05-10,2016-05-18,Ilha De Santa Maria,0,0,0,0,0,0.591304,8.0,0,0,0,0,0,0,0,0,0,1


In [15]:
train = med.drop(test["index"].to_list())

In [16]:
train = train.reset_index(drop=True)

### 1. Define train X, y

In [17]:
y = train[['No_Show']]#target / response as dependent variable

X = train.drop(['Patient_ID',
                 'Appointment_ID',
                 'Scheduled_Day', 
                 'Appointment_Day',
                 'No_Show',
                 'Neighbourhood'], axis=1) #predictors as independent variables

In [18]:
train.No_Show.value_counts()

0    78207
1    12314
Name: No_Show, dtype: int64

In [19]:
y = pd.DataFrame.to_numpy(y)

### 2. Apply SMOTE on train

In [20]:
sm = SMOTE() 
X_resampled, y_resampled = sm.fit_sample(X, y.ravel())

In [21]:
# describes info about train and test set 
print("Number transactions X_resampled dataset: ", X_resampled.shape) 
print("Number transactions y_resampled dataset: ", y_resampled.shape) 

Number transactions X_resampled dataset:  (156414, 17)
Number transactions y_resampled dataset:  (156414,)


In [22]:
np.unique(y_resampled, return_counts=True)

(array([0, 1]), array([78207, 78207], dtype=int64))

### 3. Apply KFold on train X

In [23]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics

In [24]:
kf = KFold(shuffle=True)

In [25]:
for TRAIN, TEST in kf.split(X_resampled):
    print("%s %s" % (train, test))

            Patient_ID  Appointment_ID  Gender Scheduled_Day Appointment_Day  \
0       29872499824296         5642903       1    2016-04-29      2016-04-29   
1      558997776694438         5642503       0    2016-04-29      2016-04-29   
2        4262962299951         5642549       1    2016-04-29      2016-04-29   
3         867951213174         5642828       1    2016-04-29      2016-04-29   
4        8841186448183         5642494       1    2016-04-29      2016-04-29   
...                ...             ...     ...           ...             ...   
90516    2572134369293         5651768       1    2016-05-03      2016-06-07   
90517    3596266328735         5650093       1    2016-05-03      2016-06-07   
90518   15576631729893         5630692       1    2016-04-27      2016-06-07   
90519   92134931435557         5630323       1    2016-04-27      2016-06-07   
90520  377511518121127         5629448       1    2016-04-27      2016-06-07   

           Neighbourhood  Scholarship  

            Patient_ID  Appointment_ID  Gender Scheduled_Day Appointment_Day  \
0       29872499824296         5642903       1    2016-04-29      2016-04-29   
1      558997776694438         5642503       0    2016-04-29      2016-04-29   
2        4262962299951         5642549       1    2016-04-29      2016-04-29   
3         867951213174         5642828       1    2016-04-29      2016-04-29   
4        8841186448183         5642494       1    2016-04-29      2016-04-29   
...                ...             ...     ...           ...             ...   
90516    2572134369293         5651768       1    2016-05-03      2016-06-07   
90517    3596266328735         5650093       1    2016-05-03      2016-06-07   
90518   15576631729893         5630692       1    2016-04-27      2016-06-07   
90519   92134931435557         5630323       1    2016-04-27      2016-06-07   
90520  377511518121127         5629448       1    2016-04-27      2016-06-07   

           Neighbourhood  Scholarship  

### 4. Cross Validation & Random Forest Model

In [26]:
rf_model = RandomForestClassifier(random_state=666, n_estimators=100)

In [27]:
# Perform 5-fold cross validation
scores = cross_validate(rf_model, X_resampled, y_resampled, cv=kf, scoring='accuracy', n_jobs=-1)
print ("Cross-validated scores: ", scores, "Mean accuracy score: ", scores['test_score'].mean())

Cross-validated scores:  {'fit_time': array([39.11582685, 38.91274929, 39.27204084, 38.86588645, 19.47980237]), 'score_time': array([1.92142677, 1.93704677, 1.90580392, 1.92142701, 1.14036012]), 'test_score': array([0.78921459, 0.7886392 , 0.7856024 , 0.78505898, 0.78444473])} Mean accuracy score:  0.7865919793161054


### 5. Cross Validation & Bernoulli Naive Bayes Model

In [28]:
bnb_model =  BernoulliNB()

In [29]:
# Perform 5-fold cross validation
scores = cross_validate(bnb_model, X_resampled, y_resampled, cv=kf, scoring='accuracy', n_jobs=-1)
print ("Cross-validated scores: ", scores, "Mean accuracy score: ", scores['test_score'].mean())

Cross-validated scores:  {'fit_time': array([0.18745637, 0.18745637, 0.18745732, 0.23432064, 0.17183256]), 'score_time': array([0.04686809, 0.03124452, 0.04686165, 0.03124189, 0.01562476]), 'test_score': array([0.672346  , 0.66902151, 0.67749257, 0.66860595, 0.67307078])} Mean accuracy score:  0.6721073622783974


### 6. Cross Validation &  Support Vector Machines Model

In [30]:
#svm_model = svm.SVC(gamma='auto')

In [31]:
# Perform 5-fold cross validation
#scores = cross_validate(svm_model, X_resampled, y_resampled, cv=kf, scoring='accuracy', n_jobs=-1)
#print ("Cross-validated scores: ", scores, "Mean accuracy score: ", scores['test_score'].mean())

### 7. Cross Validation &  Logistic Regression Model

In [32]:
lg_model = LogisticRegression(random_state=666, solver='lbfgs', max_iter=2000)

In [33]:
# Perform 5-fold cross validation
scores = cross_validate(lg_model, X_resampled, y_resampled, cv=kf, scoring='accuracy', n_jobs=-1)
print ("Cross-validated scores: ", scores, "Mean accuracy score: ", scores['test_score'].mean())

Cross-validated scores:  {'fit_time': array([4.35835934, 5.45185018, 5.03007388, 6.21729112, 3.40544653]), 'score_time': array([0.01561975, 0.01562309, 0.01561809, 0.01562095, 0.        ]), 'test_score': array([0.63935684, 0.63440207, 0.63673561, 0.63296359, 0.63742727])} Mean accuracy score:  0.6361770765417285


### 8. Fit Random Forest Model

In [40]:
rf_model_fitted = rf_model.fit(X_resampled, y_resampled.ravel())

### 9. Show Feature Importance of Random Forest Model

In [42]:
feature_importances = pd.DataFrame(rf_model_fitted.feature_importances_,
                                   index = X_resampled.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)

feature_importances

,importance
Age_Normalized,0.611296
Waiting_Days_Normalized,0.312946
SMS_Received_1,0.015461
Gender,0.011063
Hypertension,0.007428
Scholarship,0.006768
Day_Of_Week_Monday,0.005801
Diabetes,0.005343
Day_Of_Week_Tuesday,0.005297
Day_Of_Week_Wednesday,0.004962


### 10. Define test X, y

In [35]:
ytest = test[['No_Show']]#target / response as dependent variable

Xtest = test.drop(['Patient_ID',
                   'Appointment_ID',
                   'Scheduled_Day',
                   'Appointment_Day',
                   'No_Show',
                   'Neighbourhood',
                   'index'], axis=1)#predictors as independent variables

### 11. Apply Trained Random Forest Model on Test
    to predict accuracy on test set

In [43]:
print("Random Forest Accuracy Score: \n", rf_model_fitted.score(Xtest, ytest))

Random Forest Accuracy Score: 
 0.57895


In [44]:
print("Random Forest Confusion Matrix: \n", confusion_matrix(ytest,rf_model_fitted.predict(Xtest)))

Random Forest Confusion Matrix: 
 [[8202 1798]
 [6623 3377]]


### 12. Apply Trained Random Forest Model on Train

In [45]:
print("Random Forest Accuracy Score: \n", rf_model_fitted.score(X_resampled, y_resampled))

Random Forest Accuracy Score: 
 0.9473320802485711


In [46]:
print("Random Forest Confusion Matrix: \n", confusion_matrix(y_resampled,rf_model_fitted.predict(X_resampled)))

Random Forest Confusion Matrix: 
 [[74565  3642]
 [ 4596 73611]]
